## Prepared for the textbook:
-------------------------------------------------------------------
## Data Analysis for Business, Economics, and Policy
#### by Gabor BEKES and  Gabor KEZDI 
----------------------------------
#### Cambridge University Press 2021
-----------------------------------------------------------------------------------------------
#### License: Free to share, modify and use for educational purposes. Not to be used for business purposes.

In [1]:
import pandas as pd
import os
from pathlib import Path
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from plotnine import *
import sys
from patsy import dmatrices

In [2]:
path = Path(os.getcwd())

In [3]:
base_dir = path.parent.parent

In [4]:
data_in = os.path.join(str(base_dir) , "da_data_repo/airbnb/clean/")
data_out = os.path.join(str(base_dir), "da_case_studies/ch16-airbnb-random-forest/")
output = os.path.join(str(base_dir), "da_case_studies/ch16-airbnb-random-forest/output/")
func = os.path.join(str(base_dir) ,   "da_case_studies/ch00-tech-prep/")

In [5]:
sys.path.append(func)
from py_helper_functions import *

In [6]:
#########################################################################################
#
# PART I
# Loading and preparing data ----------------------------------------------
#
#########################################################################################

In [7]:
area="london"
data=pd.read_csv(data_in+"airbnb_"+area+"_workfile.csv")

In [8]:
#####################
### look at price ###
#####################
data.price.describe()
data=data.assign(ln_price=np.log(data.price))
data=data.query("price<=1000")

In [9]:
# Squares and further values to create
data=data.assign(
    n_accommodates2=lambda x: x['n_accommodates']**2,
    ln_accommodates=lambda x: np.log(x['n_accommodates']),
    ln_accommodates2=lambda x: np.log(x['n_accommodates'])**2,
    ln_beds=lambda x: np.log(x['n_beds']),
    ln_number_of_reviews=lambda x: np.log(x['n_number_of_reviews']+1)
)

/usr/local/lib/python3.8/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log


In [10]:
bins = pd.IntervalIndex.from_tuples([(0, 1), (1, 2), (2, 10)],closed='left')
f_bath=pd.cut(data['n_bathrooms'].to_list(), bins,labels=['0','1','2'])
f_bath.categories=[0,1,2]

In [11]:
f_bath.value_counts(dropna=False)

0        364
1      41417
2       9628
NaN      237
dtype: int64

In [12]:
# Pool accomodations with 0,1,2,10 bathrooms
data['f_bathroom']=f_bath

In [13]:
bins = pd.IntervalIndex.from_tuples([(0, 1), (1, 51), (51, max(data.n_number_of_reviews))],closed='left')
fnor=pd.cut(data['n_number_of_reviews'].to_list(), bins,labels=['0','1','2'])
fnor.categories=[0,1,2]
data['f_number_of_reviews']=fnor

In [14]:
bins = pd.IntervalIndex.from_tuples([(1, 2), (2, 3), (3, max(data.n_minimum_nights))],closed='left')
fnor=pd.cut(data['n_minimum_nights'].to_list(), bins,labels=['1','2','3'])
fnor.categories=[1,2,3]
data['f_minimum_nights']=fnor

In [15]:
# Change Infinite values with NaNs
data=data.replace([np.inf, -np.inf], np.nan)

In [16]:
data.head()

,usd_cleaning_fee,f_property_type,f_room_type,f_cancellation_policy,f_bed_type,f_neighbourhood_cleansed,usd_price_day,p_host_response_rate,n_accommodates,n_bathrooms,...,property_type,ln_price,n_accommodates2,ln_accommodates,ln_accommodates2,ln_beds,ln_number_of_reviews,f_bathroom,f_number_of_reviews,f_minimum_nights
0,NaN,Apartment,Private room,flexible,Real Bed,Kingston upon Thames,23.0,100.0,1,1.0,...,Apartment,3.135494,1,0.000000,0.000000,0.0,0.693147,1,1,3
1,NaN,Apartment,Private room,moderate,Couch,Kingston upon Thames,50.0,100.0,2,1.0,...,Apartment,3.912023,4,0.693147,0.480453,0.0,2.772589,1,1,1
2,NaN,Apartment,Private room,flexible,Real Bed,Kingston upon Thames,24.0,NaN,2,1.0,...,Apartment,3.178054,4,0.693147,0.480453,0.0,1.098612,1,1,1
3,NaN,House,Private room,flexible,Real Bed,Kingston upon Thames,50.0,NaN,2,1.5,...,House,3.912023,4,0.693147,0.480453,0.0,0.000000,1,0,2
4,NaN,House,Private room,flexible,Real Bed,Kingston upon Thames,25.0,100.0,1,1.0,...,House,3.218876,1,0.000000,0.000000,0.0,0.000000,1,0,1


In [17]:
#------------------------------------------------------------------------------------------------
# where do we have missing variables now?
to_filter=data.isna().sum()
to_filter[to_filter>0]

usd_cleaning_fee          20017
p_host_response_rate      12868
n_bathrooms                 237
n_review_scores_rating    16501
n_reviews_per_month       15741
n_beds                      167
n_days_since              15741
ln_beds                     168
f_bathroom                  237
f_number_of_reviews           1
f_minimum_nights              1
dtype: int64

In [18]:
# what to do with missing values? 
# 1. drop if no target
data = data.query('price.notna()',engine='python')

In [19]:
# 2. imput when few, not that important
data = data.assign(
    n_bathrooms = data['n_bathrooms'].fillna(np.median(data['n_bathrooms'].dropna())),
    n_beds = [x[0] if pd.isnull(x[1]) else x[1] for x in zip(data.n_accommodates,data.n_beds)],
    f_bathroom =  data['f_bathroom'].fillna(1),
    f_minimum_nights =  data['f_minimum_nights'].fillna(1),
    f_number_of_reviews = data['f_number_of_reviews'].fillna(1),
    ln_beds =  data['ln_beds'].fillna(0)
  ) 


In [20]:
data.head()

,usd_cleaning_fee,f_property_type,f_room_type,f_cancellation_policy,f_bed_type,f_neighbourhood_cleansed,usd_price_day,p_host_response_rate,n_accommodates,n_bathrooms,...,property_type,ln_price,n_accommodates2,ln_accommodates,ln_accommodates2,ln_beds,ln_number_of_reviews,f_bathroom,f_number_of_reviews,f_minimum_nights
0,NaN,Apartment,Private room,flexible,Real Bed,Kingston upon Thames,23.0,100.0,1,1.0,...,Apartment,3.135494,1,0.000000,0.000000,0.0,0.693147,1,1,3
1,NaN,Apartment,Private room,moderate,Couch,Kingston upon Thames,50.0,100.0,2,1.0,...,Apartment,3.912023,4,0.693147,0.480453,0.0,2.772589,1,1,1
2,NaN,Apartment,Private room,flexible,Real Bed,Kingston upon Thames,24.0,NaN,2,1.0,...,Apartment,3.178054,4,0.693147,0.480453,0.0,1.098612,1,1,1
3,NaN,House,Private room,flexible,Real Bed,Kingston upon Thames,50.0,NaN,2,1.5,...,House,3.912023,4,0.693147,0.480453,0.0,0.000000,1,0,2
4,NaN,House,Private room,flexible,Real Bed,Kingston upon Thames,25.0,100.0,1,1.0,...,House,3.218876,1,0.000000,0.000000,0.0,0.000000,1,0,1


In [21]:
data=data.drop(["usd_cleaning_fee", "p_host_response_rate"],axis=1)

In [22]:
to_filter=data.isna().sum()
to_filter[to_filter>0]

n_review_scores_rating    16501
n_reviews_per_month       15741
n_days_since              15741
dtype: int64

In [23]:
data

,f_property_type,f_room_type,f_cancellation_policy,f_bed_type,f_neighbourhood_cleansed,usd_price_day,n_accommodates,n_bathrooms,n_review_scores_rating,n_number_of_reviews,...,property_type,ln_price,n_accommodates2,ln_accommodates,ln_accommodates2,ln_beds,ln_number_of_reviews,f_bathroom,f_number_of_reviews,f_minimum_nights
0,Apartment,Private room,flexible,Real Bed,Kingston upon Thames,23.0,1,1.0,100.0,1,...,Apartment,3.135494,1,0.000000,0.000000,0.000000,0.693147,1,1,3
1,Apartment,Private room,moderate,Couch,Kingston upon Thames,50.0,2,1.0,91.0,15,...,Apartment,3.912023,4,0.693147,0.480453,0.000000,2.772589,1,1,1
2,Apartment,Private room,flexible,Real Bed,Kingston upon Thames,24.0,2,1.0,80.0,2,...,Apartment,3.178054,4,0.693147,0.480453,0.000000,1.098612,1,1,1
3,House,Private room,flexible,Real Bed,Kingston upon Thames,50.0,2,1.5,NaN,0,...,House,3.912023,4,0.693147,0.480453,0.000000,0.000000,1,0,2
4,House,Private room,flexible,Real Bed,Kingston upon Thames,25.0,1,1.0,NaN,0,...,House,3.218876,1,0.000000,0.000000,0.000000,0.000000,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51692,Apartment,Private room,strict,Real Bed,Westminster,24.0,1,1.0,40.0,1,...,Apartment,3.178054,1,0.000000,0.000000,0.000000,0.693147,1,1,3
51693,House,Entire home/apt,flexible,Real Bed,Richmond upon Thames,120.0,6,1.0,NaN,0,...,House,4.787492,36,1.791759,3.210402,1.791759,0.000000,1,0,3
51694,Apartment,Entire home/apt,flexible,Real Bed,Wandsworth,215.0,6,3.0,NaN,0,...,Apartment,5.370638,36,1.791759,3.210402,1.609438,0.000000,2,0,3
51695,Apartment,Entire home/apt,flexible,Real Bed,Hounslow,150.0,4,2.0,NaN,0,...,Apartment,5.010635,16,1.386294,1.921812,0.693147,0.000000,2,0,3


In [24]:
# 4. Replace missing variables re reviews with zero, when no review + add flags
data=data.assign(
    flag_days_since=np.multiply(data.n_days_since.isna(),1),
    n_days_since=data.n_days_since.fillna(np.median(data.n_days_since.dropna())),
    flag_review_scores_rating=np.multiply(data.n_review_scores_rating.isna(),1),
    n_review_scores_rating=data.n_review_scores_rating.fillna(np.median(data.n_review_scores_rating.dropna())),
    flag_reviews_per_month=np.multiply(data.n_reviews_per_month.isna(),1),
    n_reviews_per_month=data.n_reviews_per_month.fillna(np.median(data.n_reviews_per_month.dropna())),
    flag_n_number_of_reviews=np.multiply(data.n_number_of_reviews.isna(),1)
)

In [25]:
data.flag_days_since.value_counts()

0    35905
1    15741
Name: flag_days_since, dtype: int64

In [29]:
# redo features
# Create variables, measuring the time since: squared, cubic, logs
data=data.assign(
    ln_days_since =lambda x: np.log(x['n_days_since']+1),
    ln_days_since2 =lambda x:  np.log(x['n_days_since']+1)**2,
    ln_days_since3 =lambda x:  np.log(x['n_days_since']+1)**3 ,
    n_days_since2=data.n_days_since**2,
    n_days_since3=data.n_days_since**3,
    ln_review_scores_rating=np.log(data.n_review_scores_rating)
)

In [30]:
data.ln_days_since=data.ln_days_since.fillna(0)
data.ln_days_since2=data.ln_days_since2.fillna(0)
data.ln_days_since3=data.ln_days_since3.fillna(0)

In [31]:
to_filter=data.isna().sum()
to_filter[to_filter>0]

Series([], dtype: int64)

In [32]:
data.to_csv(data_out+"airbnb_london_workfile_adj.csv",index=False)